### 第5週の1日目へようこそ

Autogen AgentChat！

CrewとOpenAI Agent SDKと多くの共通点があるため、これはシンプルで馴染みがあるはずです

In [2]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

### 最初の概念：モデル

In [3]:
# LLMの初期化 OpenAI
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [4]:
# LLMの初期化 Ollama
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(model="llama3.2")

### 2番目の概念：メッセージ

In [5]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 9, 4, 31, 7, 174501, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

### 3番目の概念：エージェント

In [6]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

### on_messagesですべて？をまとめます
- LLMを設定したagentにメッセージを入力し、応答を出力する。
- CancellationTokenは、非同期処理や長時間実行されるタスクで用いられる概念

In [7]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

'Great choice! London is where the tea is always hot and the rain is always... well, also pretty much always hot!'

### チケット価格のローカルDB（SQLite）を作成

In [8]:
import os
import sqlite3

# 既存のSQLite・DBファイルが存在する場合は削除
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# データベースとテーブルを作成
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

#### DBに都市の旅費を登録するメソッド

In [9]:
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# 都市の旅費を登録
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

#### DBの都市の旅費を参照するメソッド

In [10]:
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

# Romeへの旅費を取得
get_city_price("Rome")

499.0

### 賢いエージェント

#### ...はツールを使う

In [11]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price], # ★
    reflect_on_tool_use=True
)

#### ...を実際に実行してみる。

In [12]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='call_owubkEAwNZW2Tveaw66ydyL0', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='call_owubkEAwNZW2Tveaw66ydyL0', is_error=False)]


'A roundtrip ticket to London is just $299! That’s only slightly more than what it costs to bribe a pigeon to deliver your lunch! 🕊️🍔'